In [38]:
import pytumblr
import pandas as pd
from pandas.io.json import json_normalize

In [39]:
# Authenticate via OAuth
# We only need the consumer key to call "posts" function
client = pytumblr.TumblrRestClient('nAvaCgNT6dVls4dxKYnWyM1as57L0aSAkSXAayRCPEtNxJSQjr')

In [46]:
seed_blog = 'adriwong'

for i in range(6):
    posts_json = client.posts(seed_blog + '.tumblr.com')
    posts_df = json_normalize(posts_json['posts'])
    sources_df = posts_df['source_title']

[{'type': 'photo',
  'blog_name': 'adriwong',
  'blog': {'name': 'adriwong',
   'title': 'Utopia',
   'description': '',
   'url': 'https://adriwong.tumblr.com/',
   'uuid': 't:o9aSB8pgdi8Ymb4UYS25Jw',
   'updated': 1544328217},
  'id': 111504057203,
  'post_url': 'https://adriwong.tumblr.com/post/111504057203',
  'slug': '',
  'date': '2015-02-19 22:25:08 GMT',
  'timestamp': 1424384708,
  'state': 'published',
  'format': 'html',
  'reblog_key': '1X6tIktR',
  'tags': [],
  'short_url': 'https://tmblr.co/ZPbY1w1dsAEDp',
  'summary': '',
  'should_open_in_legacy': True,
  'recommended_source': None,
  'recommended_color': None,
  'note_count': 17197,
  'source_url': 'https://studioghifli.tumblr.com/post/110997333481',
  'source_title': 'studioghifli',
  'caption': '',
  'reblog': {'comment': '', 'tree_html': ''},
  'trail': [],
  'photoset_layout': '11',
  'photos': [{'caption': '',
    'original_size': {'url': 'https://66.media.tumblr.com/418f7aacd8634f6125ba56cdc35471ce/tumblr_njruo4

0          studioghifli
1     theartofanimation
2                   NaN
3               mukuros
4                   NaN
5                   NaN
6               inluces
7               amy-box
8            sh-izukani
9              tsuiteru
10           bekki-chan
11                  NaN
12               akkise
13                phazn
14        shadow-kat-13
15            fuyuuyume
16          shimizuchan
17                  NaN
18                  NaN
19                  NaN
Name: source_title, dtype: object